In [33]:
import os 
groq_api_key=os.getenv('groq_api_key')
os.environ["HF_TOKEN"] = os.getenv('HF_TOKEN')

In [34]:
from langchain_groq import ChatGroq
llm=ChatGroq(model_name="llama-3.3-70b-versatile",groq_api_key=groq_api_key)
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002D1FA66EF90>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002D1DE3B7D10>, model_name='llama-3.3-70b-versatile', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [35]:
llm.invoke('hi').content

"It's nice to meet you. Is there something I can help you with or would you like to chat?"

In [36]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.runnables import RunnableSequence

In [ ]:
system_prompt = """
You are a smart and helpful AI assistant working for a top marketing agency in Pakistan.

Your tasks include:
- Sharing the latest marketing and advertising trends (local and global)
- Answering FAQs about services, pricing, and processes
- Assisting users in booking product/service demos
- Recommending suitable marketing services and packages
- Providing detailed product/service information
- Helping generate qualified leads
- Offering information about free trials and ongoing promotions
- Supporting other marketing-related inquiries
- if a person ask question other than marketing deny to reply

Always respond in a friendly, professional tone. Use local context (Pakistan market) where relevant. Keep answers concise, clear, and helpful. If needed, suggest actions like "Book a demo", "Request a quote", or "Start a free trial".

"\n\n"
"{context}"
"""

from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}"),
])


In [ ]:
# prompt = ChatPromptTemplate.from_messages([
#     ("system", system_prompt),
#     ("human", "{input}"),
# ])

# chat_chain = RunnableSequence(prompt | llm)

# user_input = "what the best place for digital marketing"
# response = chat_chain.invoke({"input": user_input})

# print(response.content)


In [42]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(model_name="BAAI/bge-large-en-v1.5")
vector_store = FAISS.load_local("faiss_digital_marketing_knowledgebase", embedding_model,allow_dangerous_deserialization=True)


In [43]:
retriever=vector_store.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002D1DE3B58E0>, search_kwargs={})

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [ ]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

In [44]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

question_answer_chain=create_stuff_documents_chain(llm,prompt)
rag_chain=create_retrieval_chain(retriever,question_answer_chain)

In [48]:
response=rag_chain.invoke({"input":"what is digital marketing ?"})
response['answer']

'Digital marketing, also known as online marketing, is the promotion of brands, products, or services using digital channels such as search engines, social media platforms, email, and websites. It involves using digital tools and techniques to reach, engage, and convert target audiences into customers.\n\nDigital marketing encompasses a wide range of activities, including:\n\n1. **Search Engine Optimization (SEO)**: Optimizing website content to rank higher in search engine results pages (SERPs) for specific keywords.\n2. **Pay-Per-Click (PPC) Advertising**: Creating and publishing online ads that are paid for each time a user clicks on them.\n3. **Social Media Marketing**: Using social media platforms to promote products, services, or brands, and engage with target audiences.\n4. **Content Marketing**: Creating and distributing valuable, relevant, and consistent content to attract and retain a clearly defined audience.\n5. **Email Marketing**: Sending targeted and personalized message